### Necessary Imports  and Required Jupyter Notebooks

In [1]:
import socket
import random
import time
import torch

In [2]:
%run Agent1.ipynb



### Game Initialization

We start by defining the size of the game, such as `"A4x4"` for a 4x4 board. The game size determines the complexity and the number of possible moves at each stage.

### Connection Setup

To interact with the game environment, we establish a connection to a server using socket programming. The server coordinates the flow of the game by sending the initial game state and responding to the moves made by the AlphaZero agent.

### Agent Gameplay Loop

Once connected, the agent enters a gameplay loop where it alternates between making moves and receiving responses from the server. The agent uses its `make_move` function to decide on the best action based on the current game state. This function internally calls the `selfPlay` method of the AlphaZero algorithm, which simulates games to determine the most promising move.

The moves are sent to the server in a specified format, and the server's responses are processed to update the game state. The agent continues this loop until a terminal state is reached, signified by an `"END"` message from the server, indicating either a win, loss, or draw.

### Game Conclusion

At the end of the game, the agent closes the socket connection and prints the outcome of the game. If the agent is the last one to make a valid move before the game ends, it declares victory.

This setup provides a framework for developing and testing AI agents for turn-based strategy games. The AlphaZero agent's performance can be further improved by training the underlying neural network on a dataset of high-quality game states and moves.


In [3]:
# Define the game size, this variable will be used to determine the game board dimensions.
Game="A4x4" # Options include "A6x6", "G7x7", "G9x9", "A5x5"

# Function to generate a random move for the Attax game, considering a single move.
def generate_random_move():
    x = random.randint(0, 9)
    y = random.randint(0, 9)
    return f"MOVE {x},{y}"

# Function to generate a random move for the Attax game, considering a move from one position to another.
def generate_random_move2():
    x = random.randint(0, 9)
    y = random.randint(0, 9)
    x2 = random.randint(0, 9)
    y2 = random.randint(0, 9)
    return f"MOVE {x},{y},{x2},{y2}"

# Function to connect to the game server using sockets.
def connect_to_server(host='localhost', port=12345):
    # Initialize the client socket.
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # Establish the connection with the server.
    client_socket.connect((host, port))
    # Flag to determine if the game is finished.
    flag=False
    # Receive the initial response from the server.
    response = client_socket.recv(1024).decode()
    print(f"Server ResponseINIT: {response}")
    
    # Extract the game type from the server response.
    Game = response[-4:]
    print("Playing:", Game)    

    # Initialize the AlphaZeroAgent with the given game type.
    agent=AlphaZeroAgent(Game)

    # Determine the agent number based on the server response.
    if "1" in response:
        ag=1
    else:
        ag=2

    # Variables to control the game loop.
    first=True
    run=True

    # Game loop to handle move making and response processing.
    while run:
        # Check if it's the agent's turn to play.
        if ag == 1 or not first:    
            # Make a move using the AlphaZero algorithm.
            move = agent.make_move(ag)
            time.sleep(1)
            
            # Format the move to send to the server.
            formatted_move = f"MOVE {move.xi},{move.yi},{move.xf},{move.yf}"
            # Send the formatted move to the server.
            client_socket.send(formatted_move.encode())
            print("Send:", formatted_move)
        
            # Wait for the server's response to the move.
            response = client_socket.recv(1024).decode()
            print(f"Server Response1: {response}")
            # Check if the game has ended.
            if "END" in response: break
         
        # Toggle the first-move flag after the first iteration.
        first=False
        # Receive the next response from the server.
        response = client_socket.recv(1024).decode()
        print(f"Server Response2: {response}")
        # Process the server's response and update the game state.
        run = agent.receive_response(response, ag)  

        # Check if the game has ended and set the winner flag.
        if(run==False):
            flag=True
            print("winner adversario")
        if "END" in response: break

    # Check if the agent has won and print the result.
    if(flag==False):
        print("Winner")
    # Close the socket connection.
    client_socket.close()

# Run the connect_to_server function if the script is the main program.
if __name__ == "__main__":
    connect_to_server()


Server ResponseINIT: AG1 A4x4
Playing: A4x4
Send: MOVE 0,0,2,1
Server Response1: VALID
Server Response2: MOVE 0,0,2,1
Send: MOVE 0,0,2,1
Server Response1: VALID
Server Response2: MOVE 0,0,2,1
Send: MOVE 0,0,2,1
Server Response1: VALID
Server Response2: MOVE 0,0,2,1
Send: MOVE 0,0,2,1
Server Response1: VALID
Server Response2: MOVE 0,0,2,1
Send: MOVE 0,0,2,1
Server Response1: VALID
Server Response2: END 0 10 10
Winner
